In [1]:
import os
import sys
import time
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
sys.path.append('/Users/yantingting/PycharmProjects/modeling/')
import database_conncet as data_con
import data_processing as dp
import summary_statistics as ss
from data_io_utils import *
from matplotlib.pyplot import MultipleLocator
from jinja2 import Template

In [3]:
file_path = '/Users/yantingting/Documents/用户画像(印尼)/用户分层/数据处理/'
if not os.path.exists(file_path):
    os.makedirs(file_path)
    
result_path = '/Users/yantingting/Seafile/风控/模型/10 印尼/用户画像/用户回访'

# 分层数据上传

In [23]:
df_all = load_data_from_pickle(file_path,'所有标签数据.pkl')
df_all.shape
df_all.head()

(3254826, 6)

,customer_id,type,tag_flag,freq,risk,monetary
0,141225204082151424,注册未申请,未验证身份信息,不活跃,0,0
1,141230974207135744,注册未申请,未验证身份信息,不活跃,0,0
2,141233437656088576,注册未申请,未验证身份信息,不活跃,0,0
3,141238704447815680,注册未申请,未验证身份信息,不活跃,0,0
4,141239155218063360,注册未申请,验证身份信息,不活跃,0,0


In [24]:
dict1 = {'注册未申请':1,
        '申请未放款':2,
         '新客户':3,
        '老客户':4}
dict2 = {'未验证身份信息': 1,
'验证身份信息':      2,
'申请被拒绝':       3,
'逾期未还':        4,
'其他'       :    5,
'申请取消'    :     6,
'订单进行中'  :     7,
'逾期结清'    :    8,
'提前结清'   :   9,
'正常结清'    :  10,
'申请其他'  : 11}
df_all['type'] = df_all['type'].apply(lambda x: dict1.get(x))
df_all['tag_flag'] = df_all['tag_flag'].apply(lambda x: dict2.get(x))
print(1)

1


In [49]:
import sklearn
from sklearn.model_selection import train_test_split
df_temp = df_all[(df_all['type']==1)&(df_all['tag_flag']==1)&(df_all['freq']=='不活跃')]
#dat为数据集,含有feature和label.
df_temp1, df_temp2 = train_test_split(df_temp, test_size = 0.5, random_state=1)
df_temp1.shape
df_temp2.shape

(624967, 6)

(624968, 6)

In [52]:
df_temp = df_temp2
table_sql = '''
    CREATE TABLE ytt_yhfc_1111(
        customer_id bigint,
        type text,
        tag_flag text,
        freq text,
        risk text,
        monetary text
    )
    '''
Filename = df_temp
SQL_CREATE_TABLE = table_sql
data_con.upload_df_to_pg(SQL_CREATE_TABLE)
insert = """
INSERT INTO ytt_yhfc_1111
   VALUES
 {% for var in var_list %}
{{ var }},
{% endfor %}
"""
var_list = []
for cols, rows in Filename.iterrows():    
    c = tuple(rows)
    var_list.append(c)
insert_sql = Template(insert).render(var_list=var_list)[:-2]
insert_sql = insert_sql.replace('\n','')
print(1)
data_con.upload_df_to_pg(insert_sql)

'success'

1


'success'

# 回访用户提取

In [4]:
writer = pd.ExcelWriter(os.path.join(result_path,'回访样本.xlsx'))

In [5]:
query0 = '''
select t1.*,t3.create_time as reg_time,t3.cell_phone,
t2.*
from (select *
from ytt_yhfc_4) t1 
left join 
(select *
from (
select customer_id,apply_time,
"grouping",request_principal,request_period,loan_status,
row_number()over(partition by customer_id order by effective_date desc )  as loan_rank
from rt_t_gocash_core_loan 
where effective_date !='1970-01-01') t 
where loan_rank =1) t2 
on t1.customer_id = t2.customer_id
left join rt_t_gocash_core_customer t3 
on t1.customer_id = t3.id
where loan_status !='COLLECTION' and "grouping" like 'One%'
order by t3.create_time desc 
'''

In [6]:
df_old = data_con.database_gn(query0)
# save_data_to_pickle(df_old,file_path,'老客户.pkl')
df_old.shape
df_old.head()

Successful connection


(42761, 15)

,customer_id,type,tag_flag,freq,risk,monetary,reg_time,cell_phone,customer_id,apply_time,grouping,request_principal,request_period,loan_status,loan_rank
0,436154833794662400,4,5,活跃,高风险,低收益,2020-02-17 12:23:36,089674665782,436154833794662400,2020-02-18 07:47:27,OneTest6,1700000.0,8,FUNDED,1
1,435878659763896320,4,5,活跃,高风险,低收益,2020-02-16 18:06:11,081288454949,435878659763896320,2020-02-18 17:35:18,OneTest5,1800000.0,29,FUNDED,1
2,435870295155712000,4,5,活跃,高风险,低收益,2020-02-16 17:32:56,081932533101,435870295155712000,2020-02-18 10:28:24,OneTest5,2000000.0,15,ADVANCE_PAIDOFF,1
3,435822698210697216,4,5,活跃,高风险,低收益,2020-02-16 14:23:48,082129233450,435822698210697216,2020-02-19 14:52:54,OneTest4,1800000.0,22,FUNDED,1
4,435467793960706048,4,5,活跃,高风险,低收益,2020-02-15 14:53:33,087889501725,435467793960706048,2020-02-18 13:45:30,OneTest5,1100000.0,8,FUNDED,1


In [7]:
# 低风险高收益
# 进行中未选择最高额度1500
# 结清1500
# 低风险低收益


In [8]:
df_old['是否选择最高额度'] = df_old['request_principal'].apply(lambda x: '否' if x<=1000000 else '是')
df_old1 = df_old[(df_old['risk'] == '低风险')&(df_old['monetary'] == '高收益')&(df_old['loan_status'] !='FUNDED')].head(1500)
df_old2 = df_old[(df_old['risk'] == '低风险')&(df_old['monetary'] == '高收益')
                 &(df_old['loan_status'] =='FUNDED')&(df_old['是否选择最高额度']=='否')].head(1500)
df_old3 = df_old[(df_old['risk'] == '低风险')&(df_old['monetary'] == '低收益')&(df_old['loan_status'] !='FUNDED')].head(1500)
df_old4 = df_old[(df_old['risk'] == '低风险')&(df_old['monetary'] == '低收益')
                 &(df_old['loan_status'] =='FUNDED')&(df_old['是否选择最高额度']=='否')].head(1500)
remain_list = ['customer_id','reg_time','cell_phone','是否选择最高额度']
df_old_all = pd.concat([df_old1[remain_list],df_old2[remain_list],df_old3[remain_list],df_old4[remain_list]],axis = 0)
df_old_all.shape

(4854, 5)

In [9]:
df_old_all.to_excel(writer,sheet_name='老客户')
print(1)

1


In [10]:
query_new = '''
select t1.*,
t2.cell_phone,t2.create_time as reg_time
from
(select customer_id,apply_time,effective_date,loan_status,paid_off_time,"grouping",request_principal,request_period,
case when date(paid_off_time) != '1970-01-01' then  current_date - date(paid_off_time) else null end as last_paid_now
from rt_t_gocash_core_loan
where customer_id in 
(select customer_id
from rt_t_gocash_core_loan
where effective_date !='1970-01-01'
group by customer_id
having count(1) = 1) 
and effective_date !='1970-01-01') t1 
left join rt_t_gocash_core_customer t2 
on t1.customer_id = t2.id
order by t2.create_time desc 
'''

In [11]:
df = data_con.database_gn(query_new)
df.shape
df.head()

Successful connection


(158885, 11)

,customer_id,apply_time,effective_date,loan_status,paid_off_time,grouping,request_principal,request_period,last_paid_now,cell_phone,reg_time
0,439062844380971008,2020-02-25 13:18:45,2020-02-25,FUNDED,None,Test5,1500000.0,29,NaN,087762457162,2020-02-25 12:58:59
1,439061349841600512,2020-02-25 12:59:33,2020-02-25,FUNDED,None,Test8,1500000.0,22,NaN,087731392759,2020-02-25 12:53:03
2,439060590970183680,2020-02-25 12:55:47,2020-02-25,FUNDED,None,Test5,1500000.0,29,NaN,081367543607,2020-02-25 12:50:02
3,439060496019718144,2020-02-25 13:05:38,2020-02-25,FUNDED,None,Test8,1500000.0,22,NaN,085255319025,2020-02-25 12:49:40
4,439060162090016768,2020-02-25 12:56:50,2020-02-25,FUNDED,None,Test6,1200000.0,8,NaN,087719889000,2020-02-25 12:48:20


In [ ]:
# 提前结清 
# 正常结清 
# 订单进行中
# # 未选择最高额度的；
# # 未选择最长期限的；
# # 选择最高额度的
# 逾期未还

# Test1	100万，15
# Test2	100万，22
# Test3	150万，15
# Test4	150万，22
# Test5	150万，29
# Test6	120万，22
# Test7	150万，15
# Test8	150万，22


In [12]:
df1 = df[(df['loan_status'] =='ADVANCE_PAIDOFF')&(df['last_paid_now']>=30)]
df1.shape
df2 = df[(df['loan_status'] =='PAIDOFF')&(df['last_paid_now']>=30)]
df2.shape
df3 = df[(df['loan_status'] =='FUNDED')]
df3['是否选择最高额度'] = df3.apply(lambda x:'是' if (x['grouping'] =='Test1') & (x['request_principal']>=1000000)
                                       else '是' if (x['grouping'] =='Test2') & (x['request_principal']>=1000000)
                           else '是' if (x['grouping'] =='Test3') & (x['request_principal']>=1500000)
                           else '是' if (x['grouping'] =='Test4') & (x['request_principal']>=1500000)
                           else '是' if (x['grouping'] =='Test5') & (x['request_principal']>=1500000)
                           else '是' if (x['grouping'] =='Test6') & (x['request_principal']>=1200000)
                           else '是' if (x['grouping'] =='Test7') & (x['request_principal']>=1500000)
                           else '是' if (x['grouping'] =='Test8') & (x['request_principal']>=1500000)
                           else '是' if (x['grouping'] =='ControlGroup') & (x['request_principal']>=1000000)
                           else '是' if (x['grouping'] =='') & (x['request_principal']>=1000000)
                           else '否', axis = 1)
df3['是否选择最长期限']=df3.apply(lambda x:'是' if (x['grouping'] =='Test1') & (x['request_period']>=15)
                                       else '是' if (x['grouping'] =='Test2') & (x['request_period']>=22)
                           else '是' if (x['grouping'] =='Test3') & (x['request_period']>=15)
                           else '是' if (x['grouping'] =='Test4') & (x['request_period']>=22)
                           else '是' if (x['grouping'] =='Test5') & (x['request_period']>=29)
                           else '是' if (x['grouping'] =='Test6') & (x['request_period']>=22)
                           else '是' if (x['grouping'] =='Test7') & (x['request_period']>=15)
                           else '是' if (x['grouping'] =='Test8') & (x['request_period']>=22)
                           else '是' if (x['grouping'] =='ControlGroup') & (x['request_period']>=15)
                           else '是' if (x['grouping'] =='') & (x['request_period']>=15)
                           else '否', axis = 1)
df31 = df3[df3['是否选择最高额度']=='否'].head(3000)
df31.shape
df32 = df3[df3['是否选择最长期限']=='否'].head(3000)
df32.shape
df33 = df3[df3['是否选择最高额度']=='是'].head(3000)
df33.shape
df4 = df[(df['loan_status'] =='COLLECTION')].head(3000)
df4.shape
remain_list1 = ['customer_id','reg_time','cell_phone','loan_status']
remain_list2 = ['customer_id','reg_time','cell_phone','是否选择最高额度','是否选择最长期限','loan_status']
df_new_all1 = pd.concat([df1[remain_list1],df2[remain_list1],df4[remain_list1]],axis = 0)
df_new_all1.shape
df_new_all2 = pd.concat([df31[remain_list2],df32[remain_list2],df33[remain_list2]],axis = 0)
df_new_all2.shape

(2464, 11)

(1546, 11)

/Users/yantingting/anaconda3/envs/modeling/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/Users/yantingting/anaconda3/envs/modeling/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(3000, 13)

(3000, 13)

(3000, 13)

(3000, 11)

(7010, 4)

(9000, 4)

In [13]:
df_new_all1.to_excel(writer,sheet_name='新客户(结清&逾期)')
df_new_all2.to_excel(writer,sheet_name='新客户(进行中)')
print(1)

1


In [14]:
query2 = '''
select t1.*,t3.create_time as reg_time,t3.cell_phone,
t2.*
from (select customer_id,'申请未放款' as type,
'申请取消' as tag_flag
from ytt_yhfc_2
where tag_flag = '6'
order by customer_id desc) t1 
left join 
(select *
from (
select customer_id,apply_time,
"grouping",request_principal,request_period,loan_status,
row_number()over(partition by customer_id order by apply_time desc )  as loan_rank
from rt_t_gocash_core_loan ) t 
where loan_rank =1) t2 
on t1.customer_id = t2.customer_id
left join rt_t_gocash_core_customer t3 
on t1.customer_id = t3.id
where loan_status = 'RESCIND'
order by t3.create_time desc
'''

In [15]:
df_apply = data_con.database_gn(query2)
# save_data_to_pickle(df_apply,result_path,'申请取消.pkl')
df_apply.shape

Successful connection


(56920, 12)

In [16]:
remain_list = ['customer_id','reg_time','cell_phone','loan_status']
df_apply[remain_list].head(3000).to_excel(writer,sheet_name='申请取消')
print(1)

1


In [17]:
query3 = '''
select t1.id as customer_id,t1.cell_phone,
t1.create_time,
t1.update_time,
t1.idcard_pass_flag,
'注册未申请' as type
from dw_gocash_go_cash_loan_gocash_core_customer t1 
left join 
(select distinct customer_id from dw_gocash_go_cash_loan_gocash_core_loan where date(apply_time)<='2020-02-18') t2 
on t1.id = t2.customer_id
where t2.customer_id is null and date(t1.create_time)<='2020-02-18'
order by t1.create_time desc 
limit 3000
'''

In [18]:
df_reg = data_con.database_gn(query3)
save_data_to_pickle(df_reg,result_path,'注册未申请.pkl')
df_reg.shape

Successful connection


(3000, 6)

In [19]:
remain_list = ['customer_id','create_time','cell_phone']
df_reg[remain_list].to_excel(writer,sheet_name='注册未申请')
print(1)

1


In [20]:
writer.save()
print('success')

success
